In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, Input
from tensorflow.keras.callbacks import EarlyStopping

# -----------------------------
# Step 1: Data Loading Functions
# -----------------------------
def load_combined_data(train_path, test_path):
    """
    Load combined CSV files for training and testing.
    Debugging: Prints number of rows and columns for each dataset.
    """
    try:
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)
        print(f"[DEBUG] Training data loaded: {train_df.shape[0]} rows, {train_df.shape[1]} columns")
        print(f"[DEBUG] Test data loaded: {test_df.shape[0]} rows, {test_df.shape[1]} columns")
        return train_df, test_df
    except Exception as e:
        print(f"[ERROR] Failed to load data: {e}")
        raise

def load_rul_data(file_path):
    """
    Load RUL (Remaining Useful Life) data from a text file.
    Debugging: Prints error if file cannot be loaded.
    """
    try:
        rul_df = pd.read_csv(file_path, sep=' ', header=None)
        rul_df = rul_df.dropna(axis=1)
        rul_df.columns = ['RUL']
        print(f"[DEBUG] RUL data loaded with {rul_df.shape[0]} rows.")
        return rul_df
    except Exception as e:
        print(f"[ERROR] Error loading RUL file {file_path}: {e}")
        return None

# -----------------------------
# Step 2: Feature Engineering Functions
# -----------------------------
def add_rul(df):
    """
    Calculate and add the Remaining Useful Life (RUL) for each engine unit.
    Debugging: Prints unique source files processed.
    """
    dfs = []
    unique_sources = df['source_file'].unique()
    print(f"[DEBUG] Processing RUL for source files: {unique_sources}")
    
    for source_file in unique_sources:
        df_subset = df[df['source_file'] == source_file].copy()
        # Get maximum cycle for each unit (assumed failure point in training data)
        max_cycles = df_subset.groupby('unit')['cycle'].max().reset_index()
        max_cycles.columns = ['unit', 'max_cycle']
        df_subset = df_subset.merge(max_cycles, on='unit', how='left')
        df_subset['RUL'] = df_subset['max_cycle'] - df_subset['cycle']
        df_subset.drop('max_cycle', axis=1, inplace=True)
        dfs.append(df_subset)
    
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"[DEBUG] RUL added. Data now has {combined_df.shape[1]} columns.")
    return combined_df

def add_failure_indicator(df, threshold=30):
    """
    Add a binary failure indicator: 1 if RUL <= threshold, else 0.
    Debugging: Print the distribution of the new indicator.
    """
    df['failure_imminent'] = (df['RUL'] <= threshold).astype(int)
    print(f"[DEBUG] Failure indicator added (threshold={threshold}). Distribution:\n{df['failure_imminent'].value_counts()}")
    return df

# -----------------------------
# Step 3: Preprocessing Functions
# -----------------------------
def preprocess_data(df, scaler=None, is_training=True):
    """
    Preprocess sensor and operational features:
      - Convert to numeric and scale features.
    Debugging: Print shape of scaled features.
    """
    # Identify sensor and operational setting columns
    sensor_cols = [col for col in df.columns if col.startswith('sensor')]
    op_cols = [col for col in df.columns if col.startswith('op_set')]
    feature_cols = op_cols + sensor_cols

    df_copy = df.copy()
    
    # Convert feature columns to numeric
    for col in feature_cols:
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
    
    # Scale features
    if scaler is None and is_training:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_copy[feature_cols])
        print(f"[DEBUG] Features scaled (training). Shape: {X_scaled.shape}")
    else:
        X_scaled = scaler.transform(df_copy[feature_cols])
        print(f"[DEBUG] Features scaled (inference). Shape: {X_scaled.shape}")
    
    # Build a new DataFrame with scaled features
    X_scaled_df = pd.DataFrame(X_scaled, columns=feature_cols)
    # Reattach metadata columns
    for col in ['unit', 'cycle', 'RUL', 'failure_imminent', 'source_file']:
        if col in df_copy.columns:
            X_scaled_df[col] = df_copy[col].values
    
    return (X_scaled_df, scaler) if is_training else X_scaled_df

def reduce_dimensions(df, pca=None, n_components=10, is_training=True):
    """
    Reduce dimensions using PCA on sensor and operational features.
    Debugging: Print explained variance ratio.
    """
    feature_cols = [col for col in df.columns if col.startswith('sensor') or col.startswith('op_set')]
    
    if pca is None and is_training:
        pca = PCA(n_components=n_components)
        X_pca = pca.fit_transform(df[feature_cols])
        print(f"[DEBUG] PCA fitted. Explained variance ratio: {pca.explained_variance_ratio_}")
    else:
        X_pca = pca.transform(df[feature_cols])
    
    X_pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(n_components)])
    
    # Reattach metadata columns
    for col in ['unit', 'cycle', 'RUL', 'failure_imminent', 'source_file']:
        if col in df.columns:
            X_pca_df[col] = df[col].values
    
    return (X_pca_df, pca) if is_training else X_pca_df

# -----------------------------
# Step 4: Sequence Preparation for LSTM
# -----------------------------
def prepare_sequences(df, sequence_length=30, step=1):
    """
    Convert time-series data into sequences for LSTM.
    Debugging: Print the number of sequences created.
    """
    feature_cols = [col for col in df.columns if col.startswith('PC')]
    metadata_cols = ['unit', 'cycle', 'RUL', 'failure_imminent', 'source_file']
    metadata_cols = [col for col in metadata_cols if col in df.columns]
    
    sequences = []
    metadata = []
    
    for source_file in df['source_file'].unique():
        df_dataset = df[df['source_file'] == source_file]
        for unit in df_dataset['unit'].unique():
            unit_data = df_dataset[df_dataset['unit'] == unit].sort_values('cycle')
            if len(unit_data) >= sequence_length:
                features = unit_data[feature_cols].values
                unit_metadata = unit_data[metadata_cols].values
                for i in range(0, len(features) - sequence_length + 1, step):
                    sequences.append(features[i:i+sequence_length])
                    metadata.append(unit_metadata[i+sequence_length-1])
    
    sequences_arr = np.array(sequences)
    metadata_arr = np.array(metadata)
    print(f"[DEBUG] Prepared {sequences_arr.shape[0]} sequences of length {sequence_length}.")
    return sequences_arr, metadata_arr

# -----------------------------
# Step 5: Define LSTM Autoencoder Model
# -----------------------------
class LSTMAutoencoder:
    def __init__(self, input_dim, latent_dim=16, timesteps=30):
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.timesteps = timesteps
        self.model = self._build_model()
        
    def _build_model(self):
        # Build encoder
        inputs = Input(shape=(self.timesteps, self.input_dim))
        encoded = LSTM(32, activation='relu', return_sequences=True)(inputs)
        encoded = LSTM(self.latent_dim, activation='relu')(encoded)
        
        # Build decoder
        decoded = RepeatVector(self.timesteps)(encoded)
        decoded = LSTM(32, activation='relu', return_sequences=True)(decoded)
        decoded = LSTM(self.input_dim, activation='relu', return_sequences=True)(decoded)
        
        autoencoder = Model(inputs, decoded)
        autoencoder.compile(optimizer='adam', loss='mse')
        print("[DEBUG] LSTM Autoencoder model built.")
        return autoencoder
    
    def fit(self, X, validation_data=None, epochs=50, batch_size=32):
        early_stopping = EarlyStopping(
            monitor='val_loss' if validation_data is not None else 'loss',
            patience=5,
            restore_best_weights=True
        )
        history = self.model.fit(
            X, X,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(validation_data, validation_data) if validation_data is not None else None,
            callbacks=[early_stopping],
            verbose=1
        )
        return history
    
    def predict(self, X):
        return self.model.predict(X)
    
    def compute_anomaly_scores(self, X, threshold=None):
        X_pred = self.predict(X)
        mse = np.mean(np.power(X - X_pred, 2), axis=(1, 2))
        if threshold is None:
            threshold = np.percentile(mse, 95)
        anomalies = (mse > threshold).astype(int)
        return anomalies, mse, threshold

# -----------------------------
# Step 6: Main Function – Putting Everything Together
# -----------------------------
def main():
    # File paths: update with your actual file locations
    combined_train_path = "combined_cmapps_training.csv"
    combined_test_path = "combined_cmapps_dataset.csv"
    
    print("[INFO] Loading combined datasets...")
    combined_train, combined_test = load_combined_data(combined_train_path, combined_test_path)
    
    # Check required column
    for df in [combined_train, combined_test]:
        if 'source_file' not in df.columns:
            print("[ERROR] 'source_file' column is missing. Exiting.")
            return
    
    print("[INFO] Processing training data: Adding RUL and failure indicators...")
    train_df_with_rul = add_rul(combined_train)
    train_df_with_indicators = add_failure_indicator(train_df_with_rul)
    
    print("[INFO] Preprocessing training data...")
    X_train_scaled, scaler = preprocess_data(train_df_with_indicators, is_training=True)
    
    print("[INFO] Reducing dimensions using PCA...")
    X_train_pca, pca = reduce_dimensions(X_train_scaled, n_components=10, is_training=True)
    
    sequence_length = 30
    print(f"[INFO] Preparing sequences (length={sequence_length}) for training...")
    X_sequences, X_metadata = prepare_sequences(X_train_pca, sequence_length=sequence_length)
    
    # Debug: Check sequence shapes
    print(f"[DEBUG] Training sequences shape: {X_sequences.shape}")
    
    # Split training sequences for validation (80/20 split)
    split_idx = int(0.8 * len(X_sequences))
    X_train_seq, X_val_seq = X_sequences[:split_idx], X_sequences[split_idx:]
    print(f"[DEBUG] Training set: {X_train_seq.shape}, Validation set: {X_val_seq.shape}")
    
    print("[INFO] Training LSTM Autoencoder...")
    input_dim = X_train_seq.shape[2]
    lstm_ae = LSTMAutoencoder(input_dim=input_dim, latent_dim=8, timesteps=sequence_length)
    history = lstm_ae.fit(X_train_seq, validation_data=X_val_seq, epochs=30, batch_size=32)
    
    # Save the trained model for later use
    model_save_path = 'lstm_autoencoder.h5'
    lstm_ae.model.save(model_save_path)
    print(f"[INFO] Model saved to '{model_save_path}'.")
    
    # Visualize training history
    plt.figure()
    plt.plot(history.history['loss'], label='Training Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    # Compute source-specific thresholds on validation data
    print("[INFO] Computing source-specific anomaly thresholds...")
    def compute_source_thresholds(model, sequences, metadata):
        thresholds = {}
        _, mse, _ = model.compute_anomaly_scores(sequences)
        source_file_col_idx = -1  # assuming the last column of metadata is 'source_file'
        for source_file in np.unique(metadata[:, source_file_col_idx]):
            source_indices = metadata[:, source_file_col_idx] == source_file
            source_mse = mse[source_indices]
            if len(source_mse) > 0:
                thresholds[source_file] = np.percentile(source_mse, 95)
        return thresholds

    source_thresholds = compute_source_thresholds(lstm_ae, X_val_seq, X_metadata)
    for src, thr in source_thresholds.items():
        print(f"[DEBUG] Source '{src}' threshold: {thr:.6f}")
    
    print("[INFO] Processing test data...")
    test_df_with_rul = add_rul(combined_test)
    test_df_with_indicators = add_failure_indicator(test_df_with_rul)
    X_test_scaled = preprocess_data(test_df_with_indicators, scaler=scaler, is_training=False)
    X_test_pca = reduce_dimensions(X_test_scaled, pca=pca, is_training=False)
    X_test_sequences, X_test_metadata = prepare_sequences(X_test_pca, sequence_length=sequence_length)
    print(f"[DEBUG] Created {len(X_test_sequences)} test sequences")
    
    print("[INFO] Detecting anomalies in test data...")
    all_anomalies, all_mse, _ = lstm_ae.compute_anomaly_scores(X_test_sequences)
    print(f"[RESULT] Total anomalies detected in test data: {all_anomalies.sum()} out of {len(all_anomalies)} sequences")
    
    # Visualize reconstruction error distribution on test data
    plt.figure()
    sns.kdeplot(all_mse, label='Test MSE')
    plt.title('Reconstruction Error Distribution on Test Data')
    plt.xlabel('MSE')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()
